In [1]:
import pandas as pd

In [3]:
url = "https://raw.githubusercontent.com/jakevdp/data-CDCbirths/master/births.csv"
df = pd.read_csv(url)

In [20]:
df.head()

,year,month,day,gender,births
0,1969,1,1.0,F,4046
1,1969,1,1.0,M,4440
2,1969,1,2.0,F,4454
3,1969,1,2.0,M,4548
4,1969,1,3.0,F,4548


In [15]:
df.shape

(15547, 5)

In [12]:
df.describe()

,year,month,day,births
count,15547.000000,15547.000000,15067.000000,15547.000000
mean,1979.037435,6.515919,17.769894,9762.293561
std,6.728340,3.449632,15.284034,28552.465810
min,1969.000000,1.000000,1.000000,1.000000
25%,1974.000000,4.000000,8.000000,4358.000000
50%,1979.000000,7.000000,16.000000,4814.000000
75%,1984.000000,10.000000,24.000000,5289.500000
max,2008.000000,12.000000,99.000000,199622.000000


In [18]:
df.isnull().sum()

year        0
month       0
day       480
gender      0
births      0
dtype: int64

In [24]:
# gb = df.groupby(['year', 'month', 'day'])
# df_noGender = gb.sum()
# df_noGender

In [27]:
df = df.drop('gender', axis=1)
df.head()

,year,month,day,births
0,1969,1,1.0,4046
1,1969,1,1.0,4440
2,1969,1,2.0,4454
3,1969,1,2.0,4548
4,1969,1,3.0,4548
